## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [5]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'], port=6000)
session = cluster.connect()

#### Create & Set Keyspace

In [6]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
session.execute("""
    CREATE KEYSPACE if not exists soundcloud
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }
""")

In [7]:
session.set_keyspace('soundcloud')

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [5]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command                   

In [6]:
q1='''CREATE TABLE IF NOT EXISTS song_play_history (
    session_number INT,
    item_in_session_number INT,
    artist_name TEXT,
    song_title TEXT,
    length FLOAT,
    PRIMARY KEY (session_number, item_in_session_number)
);'''
session.execute(q1)

### Let's insert our data into of table

In [7]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = """INSERT INTO song_play_history (session_number, item_in_session_number, artist_name, song_title, length)
        VALUES (%s, %s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))

### Validate our Data Model using a SELECT

In [9]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

rows = session.execute("""
SELECT artist_name, song_title, length FROM song_play_history 
WHERE session_number = 338 AND item_in_session_number = 4
""")
for row in rows:
    print(row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [66]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command                   

session.execute("""
CREATE TABLE IF NOT EXISTS user_song_play_history (
    user_id INT,
    session_number INT,
    item_in_session_number INT,
    artist_name TEXT,
    song_title TEXT,
    fname TEXT,
    lname TEXT,
    PRIMARY KEY ((user_id, session_number), item_in_session_number)
)
""")

### Let's insert our data into of table

In [67]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = '''INSERT INTO user_song_play_history (user_id, session_number, item_in_session_number, artist_name, song_title, fname, lname)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)'''
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (int(row[10]), int(row[8]), int(row[3]), row[0], row[9], row[1], row[4]))

### Validate our Data Model using a SELECT

In [72]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
result = session.execute("""
    SELECT artist_name, song_title, fname, lname
    FROM user_song_play_history
    WHERE user_id = 10 AND session_number = 182
    ORDER BY item_in_session_number;
""")

for row in result:
    print(row.artist_name, row.song_title, row.fname, row.lname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [33]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command      

session.execute("""
CREATE TABLE IF NOT EXISTS song_listeners (
    song_title TEXT,
    user_id text,
    fname TEXT,
    lname TEXT,
    PRIMARY KEY (song_title, user_id)
)
""")             

### Let's insert our data into of table

In [36]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
## Task: Write the INSERT statements and assign it to the query variable
        query = """INSERT INTO song_listeners (song_title, user_id, fname, lname) VALUES (%s, %s, %s, %s)"""
        
        ## Task: Match the column in the csv file to the column in the INSERT statement.
        ## e.g., if you want to INSERT gender from csv file into the database you will use row[2]
        ## e.g., if you want to INSERT location from csv file into database you will use row[7]
        session.execute(query, (row[9], row[10], row[1], row[4]))

### Validate our Data Model using a SELECT

In [37]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results
rows = session.execute("""SELECT fname, lname FROM song_listeners WHERE song_title = 'All Hands Against His Own'""")

# Iterate through the result rows and display names
for row in rows:
    print(f"User: {row.fname} {row.lname}")

User: Jacqueline Lynch
User: Tegan Levine
User: Sara Johnson


### Drop the tables before closing out the sessions

In [8]:
session.execute("drop table song_listeners ")

In [9]:
session.execute("drop table song_play_history")

In [10]:
session.execute("drop table user_song_play_history")

### Close the session and cluster connection¶

In [11]:
session.shutdown()

In [12]:
cluster.shutdown()